# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/DynamicHMC/iPqvI.ji for DynamicHMC [bbc10e6e-7c05-544b-b16e-64fede858acb]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.2/MCMCDiagnostics/kMBOZ.ji for MCMCDiagnostics [6e857e4b-079a-58c4-aeab-bc2670384359]
└ @ Base loading.jl:1187


Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.8075078261961934

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
1.2e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0024 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00012 s/step ...done
MCMC, adapting ϵ (200 steps)
4.1e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
8.5e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
8.3e-6 s/step ...done
MCMC (1000 steps)
7.5e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.591686], -2.83803, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([0.901785], -2.98418, 2, DoubledTurn, 0.970222, 3), NUTS_Transition{Array{Float64,1},Float64}([1.15543], -3.20483, 1, DoubledTurn, 0.935863, 1), NUTS_Transition{Array{Float64,1},Float64}([0.633432], -5.15664, 2, DoubledTurn, 0.762095, 3), NUTS_Transition{Array{Float64,1},Float64}([0.582984], -2.90947, 2, DoubledTurn, 0.978438, 3), NUTS_Transition{Array{Float64,1},Float64}([0.582984], -4.32321, 1, AdjacentTurn, 0.62261, 3), NUTS_Transition{Array{Float64,1},Float64}([1.42542], -3.85463, 1, AdjacentTurn, 0.790877, 3), NUTS_Transition{Array{Float64,1},Float64}([1.54719], -4.00617, 2, DoubledTurn, 0.985768, 3), NUTS_Transition{Array{Float64,1},Float64}([0.832398], -4.02319, 2, DoubledTurn, 0.982958, 3), NUTS_Transition{Array{Float64,1},Float64}([1.02015], -3.03934, 1, DoubledTurn, 0.96005, 1)  …  NUTS_Transit

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

388.38844801259603

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.29 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 32% DoubledTurn => 68%
  depth: 1 => 66% 2 => 34%


check the mean

In [11]:
mean(posterior_α)

0.640383411790054

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*